# Imports

In [14]:
import os
from tqdm.notebook import tqdm
import xml.etree.ElementTree as ET
import subprocess
import json

# XML Parser

In [7]:
def replace_key_in_xml(xml_file, key_val, output_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for elem in root.iter():
        if elem.tag in key_val:
            key = elem.tag
            val = key_val[key]
            if key == "L3Cache_12M_16":
                for element in elem:
                    if element.tag == "Prefetcher":
                        element.text = val
            else:
                elem.text = val
    tree.write(output_file)

# Build and Run

In [8]:
def run(file_name, benchmark_path=""):
    err_file = f"./err/{file_name}.err"
    assignment_dir="/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5"
    resource_dir = "/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources"
    cmd = f"java -jar {resource_dir}/Tejas/jars/tejas.jar {assignment_dir}/config_tigerlake_modified.xml {assignment_dir}/outputs/{file_name}.stat"
    cmd += f" {benchmark_path}"
    with open(err_file, "w") as output_file:
        print(cmd)
        subprocess.run(cmd, shell=True, stdout=output_file)

# Constants

In [9]:
prefetchers = ["Simple", "Power4", "Bingo", "None"]
config_file_path = "./config_tigerlake.xml"
output_file = "./config_tigerlake_modified.xml"

# New Benchmarks

In [10]:
benchmark_path = "/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/phoronix_short_traces/"
benchmarks = os.listdir(benchmark_path)
bad_bench = ["intel-mlc_8", "stream_8"]
benchmarks = [x for x in benchmarks if x not in bad_bench and "_8" in x]

total = len(prefetchers) * len(benchmarks)
pb = tqdm(total=total)
# for prefetcher in prefetchers:
    # for benchmark in benchmarks:
    #     pb.update(1)
    #     print(benchmark, prefetcher)
    #     key_vals = {
    #         "isVISATrace": "false",
    #         "L3Cache_12M_16": prefetcher,
    #         "BenchmarkPath": benchmark_path+benchmark+f"/{benchmark.split('_')[0]}",
    #     }
    #     replace_key_in_xml(config_file_path, key_vals, output_file)
    #     file_name = f"{benchmark}_{prefetcher}"
    #     run(file_name)

  0%|          | 0/20 [00:00<?, ?it/s]

# Old Benchmarks

In [11]:
benchmark_path = "/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/CPU2017_benchmarks/tejas_traces/"
old_benchmarks = os.listdir(benchmark_path)
old_benchmarks = list(set([x.split("_")[0] for x in old_benchmarks]))

total = len(prefetchers) * len(old_benchmarks)
pb = tqdm(total=total)
# for prefetcher in prefetchers:
#     for benchmark in old_benchmarks:
#         pb.update(1)
#         print(benchmark, prefetcher)
#         full_benchmark_path = benchmark_path+benchmark
#         key_vals = {
#             "isVISATrace": "true",
#             "L3Cache_12M_16": prefetcher,
#             "BenchmarkPath": full_benchmark_path
#         }
#         replace_key_in_xml(config_file_path, key_vals, output_file)
#         file_name = f"{benchmark}_{prefetcher}"
#         run(file_name, benchmark_path=full_benchmark_path)

  0%|          | 0/20 [00:00<?, ?it/s]

# Statistics Parser

In [12]:
def stat_parser(stat_file_path):
    avg_mipc, mc = 0, 0
    avg_cipc, cc = 0, 0
    count = 0
    i1_hit, l1_hit, l2_hit, l3_hit = [0]*4
    with open(stat_file_path, "r") as f:
        for line in f.readlines():
            if line.startswith("IPC"):
                ipc = float(line.split("=")[1].strip().split("\t")[0])
                if "micro-ops"in line:
                    avg_mipc += ipc
                    mc += 1
                if "CISC" in line:
                    avg_cipc += ipc
                    cc += 1
            if line.startswith("L3 Hit-Rate"):
                l3_hit = float(line.split("=")[1].strip().split("\t")[0])
            if line.startswith("L2 Hit-Rate"):
                l2_hit = float(line.split("=")[1].strip().split("\t")[0])
            if line.startswith("L1 Hit-Rate"):
                l1_hit = float(line.split("=")[1].strip().split("\t")[0])
            if line.startswith("I1 Hit-Rate"):
                i1_hit = float(line.split("=")[1].strip().split("\t")[0])
    return {
        "Micro Ops IPC": avg_mipc/mc,
        "CISC IPC": avg_cipc/cc,
        "I1 Hit Rate": i1_hit,
        "L1 Hit Rate": l1_hit,
        "L2 Hit Rate": l2_hit,
        "L3 Hit Rate": l3_hit
    }

# Analyze

In [19]:
results = {}
for benchmark in benchmarks+old_benchmarks:
    results[benchmark] = {}
    for prefetcher in prefetchers:
        stat_file = f"./outputs/{benchmark}_{prefetcher}.stat"
        try:
            print(benchmark, prefetcher)
            parsed_res = stat_parser(stat_file)
            results[benchmark][prefetcher] = parsed_res
        except:
            print("File Not Found:", stat_file)

with open("results.json", "w+") as f:
    json.dump(results, f, indent=4)

openssl_8 Simple
openssl_8 Power4
openssl_8 Bingo
openssl_8 None
blender_8 Simple
blender_8 Power4
blender_8 Bingo
blender_8 None
gimp_8 Simple
gimp_8 Power4
gimp_8 Bingo
gimp_8 None
numenta-nab_8 Simple
numenta-nab_8 Power4
numenta-nab_8 Bingo
numenta-nab_8 None
compress-7zip_8 Simple
File Not Found: ./outputs/compress-7zip_8_Simple.stat
compress-7zip_8 Power4
File Not Found: ./outputs/compress-7zip_8_Power4.stat
compress-7zip_8 Bingo
File Not Found: ./outputs/compress-7zip_8_Bingo.stat
compress-7zip_8 None
File Not Found: ./outputs/compress-7zip_8_None.stat
mcf Simple
mcf Power4
mcf Bingo
mcf None
namd Simple
namd Power4
namd Bingo
namd None
gcc Simple
gcc Power4
gcc Bingo
gcc None
xalancbmk Simple
xalancbmk Power4
xalancbmk Bingo
xalancbmk None
lbm Simple
lbm Power4
lbm Bingo
lbm None
